# Import libraries

In [1]:
import numpy as np
import random as rn
import os
from keras import backend as K
import  tensorflow.compat.v1  as tf
tf.disable_v2_behavior() 
import tensorflow as tf

#--------------------------------------------------------------------------------------------------------------------
# Reproducible
seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)
session_conf =tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.compat.v1.set_random_seed(seed)
#--------------------------------------------------------------------------------------------------------------------

import scipy.sparse as sp
import time
import pandas as pd
import sys

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

sys.path.append(r"../../")

from dgae_alpha.evaluation import get_roc_score
from dgae_alpha.input_data import load_data, load_label
from dgae_alpha.model import *
from dgae_alpha.optimizer import *
from dgae_alpha.preprocessing import *

Instructions for updating:
non-resource variables are not supported in the long term


# Parameters setting

In [2]:
layers_no=6
data_path="../../Datasets/"
path_now="./log/"

dataset='citeseer'
model_name='DGAE_alpha_feature'

dropout=0.0
epochs=200
features_used=True
learning_rate=0.01
nb_run=10
prop_val=5
prop_test=10
validation=True
verbose=True

# Load data

In [3]:
adj_init, features_init = load_data(dataset,data_path)
num_adj=adj_init.shape[1]

# Training and testing

In [4]:
loss_coeff_adj=0.001
loss_coeff_AE=0.1
results_output_path = path_now+model_name+"/"
mkdir(results_output_path)

for seed_i in np.arange(nb_run):
    
    seed=seed_i
    lost_list=[]
    roc_list=[]
    ap_list=[]
    mean_time=[]
    
    adj, val_edges, val_edges_false, test_edges, test_edges_false =mask_test_edges(adj_init, seed,prop_test, prop_val)
                
    t_start = time.time()

    if features_used:
        features = features_init
    
    num_nodes = adj.shape[0]

    if not features_used:
        features = sp.identity(adj.shape[0])

    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
        
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ())
    }

#--------------------------------------------------------------------------------------------------------------------
# Create model
    if model_name=='DGAE_alpha_feature':
        model = DGAE_alpha_feature(placeholders, layers_no,num_features, features_nonzero)
    elif model_name=='DGAE_alpha_no_feature':
        model=DGAE_alpha_no_feature(placeholders, layers_no,num_features, num_adj,features_nonzero)
    else:
        raise ValueError('Undefined model!')

    pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0]- adj.sum()) * 2)
    
    with tf.name_scope('optimizer'):
        if model_name in ('DGAE_alpha_feature'):
            opt =OptimizerAE_FeatureReconstrution(preds_adj = model.reconstructions,\
                                                  preds_Features=model.feature_reconstruction,\
                                                  labels_adj = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],\
                                                                                                    validate_indices = False), [-1]),\
                                                  labels_Features=placeholders['features'],\
                                                  labels_adj_rec=placeholders['adj'],\
                                                  loss_coeff_adj=loss_coeff_adj,\
                                                  loss_coeff_AE=loss_coeff_AE,\
                                                  pos_weight = pos_weight,\
                                                  norm = norm)
                
        elif model_name in ('DGAE_alpha_no_feature'):
            opt =OptimizerAE_AdjReconstrution(preds_adj = model.reconstructions,\
                                              preds_adj_rec=model.adj_reconstruction,\
                                              labels_adj = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                                                validate_indices = False), [-1]),\
                                              labels_Features=placeholders['features'],\
                                              labels_adj_rec=placeholders['adj'],\
                                              loss_coeff_adj=loss_coeff_adj,\
                                              loss_coeff_AE=loss_coeff_AE,\
                                              pos_weight = pos_weight,\
                                              norm = norm)

    adj_norm = preprocess_graph(adj)
    adj_label = sparse_to_tuple(adj + sp.eye(adj.shape[0]))

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        t = time.time()
        feed_dict = construct_feed_dict(adj_norm, adj_label, features,placeholders)
        feed_dict.update({placeholders['dropout']: dropout})
        outs = sess.run([opt.opt_op, opt.cost, opt.accuracy],feed_dict = feed_dict)
        avg_cost = outs[1]
        if verbose:
            lost_list.append(avg_cost)
            print("Epoch:", '%04d' % (epoch + 1), "training loss=", "{:.5f}".format(avg_cost),"time=", "{:.5f}".format(time.time() - t))
                    
            if validation:
                feed_dict.update({placeholders['dropout']: 0})
                emb = sess.run(model.z_mean, feed_dict = feed_dict)
                feed_dict.update({placeholders['dropout']: dropout})
                val_roc, val_ap = get_roc_score(val_edges, val_edges_false, emb)
                roc_list.append(val_roc)
                ap_list.append(val_ap)
                print("validation roc=", "{:.5f}".format(val_roc), "validation ap=", "{:.5f}".format(val_ap))

    emb = sess.run(model.z_mean, feed_dict = feed_dict)   
    mean_time.append(time.time() - t_start)
    
    roc_score, ap_score = get_roc_score(test_edges, test_edges_false, emb)
    print("testing roc=", "{:.5f}".format(roc_score), "testing ap=", "{:.5f}".format(ap_score))
    
    roc_list.append(roc_score)
    ap_list.append(ap_score)
    
    mean_time_=np.array(mean_time)
    write_to_csv(mean_time_.reshape(1,len(mean_time_)),results_output_path+"layers_"+str(layers_no)+"_time_10run.csv")

    roc_list_=np.array(roc_list)
    write_to_csv(roc_list_.reshape(1,len(roc_list_)),results_output_path+"layers_"+str(layers_no)+"_roc_10run.csv")

    ap_list_=np.array(ap_list)
    write_to_csv(ap_list_.reshape(1,len(ap_list_)),results_output_path+"layers_"+str(layers_no)+"_ap_10run.csv")

Failure
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
targets is deprecated, use labels instead
Epoch: 0001 training loss= 0.00125 time= 1.88246
validation roc= 0.64769 validation ap= 0.68269
Epoch: 0002 training loss= 0.00125 time= 0.65799
validation roc= 0.72122 validation ap= 0.70837
Epoch: 0003 training loss= 0.00121 time= 0.62060
validation roc= 0.63056 validation ap= 0.63175
Epoch: 0004 training loss= 0.00120 time= 0.59739
validation roc= 0.70787 validation ap= 0.72008
Epoch: 0005 training loss= 0.00116 time= 0.49030
validation roc= 0.79330 validation ap= 0.81099
Epoch: 0006 training loss= 0.00110 time= 0.80016
validation roc= 0.82274 validation ap= 0.83362
Epoch: 0007 training loss= 0.00105 time= 0.43019
validation roc= 0.83314 validation ap= 0.83527
Epoch: 0008 training loss= 0.00101 time= 0.56053
validation roc= 0.83918 validation ap= 0.83453
Epoch: 0009 training loss= 0.00100 time=

# Results

In [5]:
time_10run=results_output_path+"layers_"+str(layers_no)+"_time_10run.csv"
roc_10run=results_output_path+"layers_"+str(layers_no)+"_roc_10run.csv"
ap_10run=results_output_path+"layers_"+str(layers_no)+"_ap_10run.csv"

mean_time_read=np.array(pd.read_csv(time_10run,header=None))
mean_roc_read=np.array(pd.read_csv(roc_10run,header=None))
mean_ap_read=np.array(pd.read_csv(ap_10run,header=None))

p_used_number=10
time_test=mean_time_read[:,-1][0:p_used_number]
roc_test=mean_roc_read[:,-1][0:p_used_number]
ap_test=mean_ap_read[:,-1][0:p_used_number]

print("Time: ", time_test)
print("\n")
print("AUC: ", roc_test)
print("\n")
print("AP: ", ap_test)

Time:  [115.45913029 115.13654256 111.82132983 102.70911598 101.19816661
 100.82126141 100.12081432 110.42329359 108.51497006 101.72890735]


AUC:  [0.94745079 0.9441396  0.9346649  0.942007   0.94172684 0.94103611
 0.92910518 0.94571187 0.95489917 0.93575414]


AP:  [0.95183017 0.94873813 0.94634011 0.94527161 0.94723691 0.94934752
 0.94342635 0.95327438 0.95178336 0.94913366]


In [6]:
print("Averaged AUC: ", str(np.mean(roc_test))+'±'+str(np.std(roc_test)))
print("\n")
print("Averaged AP: ",  str(np.mean(ap_test))+'±'+str(np.std(ap_test)))

Averaged AUC:  0.9416495592319768±0.006879840783111509


Averaged AP:  0.94863822064394±0.002967752361405977
